In [19]:
import os
from glob import glob
from datetime import datetime, timezone
import pickle

import numpy as np
import pandas as pd
import xarray as xr

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


%matplotlib widget
import matplotlib.pyplot as plt

In [20]:
cloud_dfN = pd.read_parquet("../data/caliop-slstr-greenland-maxsza80.parquet")
print(len(cloud_dfN))

1244598


In [21]:
dates = cloud_dfN.Profile_UTC_Time

dates.min(), dates.max(), np.sum(dates.dt.year == 2019), np.sum(dates.dt.year == 2020)

(Timestamp('2019-04-01 11:29:54+0000', tz='UTC'),
 Timestamp('2020-09-30 14:44:26+0000', tz='UTC'),
 865529,
 379069)

In [22]:
cloud_dfS = pd.read_parquet("../data/caliop-slstr-antarctic-maxsza80.parquet")
print(len(cloud_dfS))

dates.min(), dates.max(), np.sum(dates < datetime(2019, 6, 1, tzinfo=timezone.utc)), \
        np.sum(dates > datetime(2019, 6, 1, tzinfo=timezone.utc))

235122


(Timestamp('2019-04-01 11:29:54+0000', tz='UTC'),
 Timestamp('2020-09-30 14:44:26+0000', tz='UTC'),
 310423,
 934175)

In [23]:
cloud_df = pd.concat((cloud_dfN, cloud_dfS))
cloud_df = cloud_df.sample(frac = 1, random_state=7273)

cloud_df = cloud_df
#cloud_df = cloud_df[cloud_df.file.str.find('S3A') >= 0]
len(cloud_df)

1479720

In [24]:

def get_xy(cloud_df):
    data = cloud_df.copy()

    data['aa'] = (data['satellite_azimuth_angle'] - data['solar_azimuth_angle']) % 360

    inputs = ['S1', 'S2', 'S3', 'S4', 'S5', 'S7', 'S8', 'S9', 'satellite_zenith_angle', 'solar_zenith_angle', 'aa']
    X = data[inputs].values

    y= cloud = cloud_df['Number_Layers_Found'] > 0
    return X, y

X, y = get_xy(cloud_df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.90)

In [25]:
tree=False

#classifier = RandomForestClassifier(n_estimators = 100)
classifier = RandomForestClassifier(n_estimators = 5, max_depth = 12, max_leaf_nodes=256) 

#classifier = DecisionTreeClassifier(max_depth = 10, max_leaf_nodes=50); tree=True
    
classifier.fit(X_train, y_train)


 
y_pred = classifier.predict(X_test)



In [26]:
date = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
with open(f"../models/caliop-random-forest-{date}.pkl", "wb") as f:
    pickle.dump(classifier, f)

with open(f"../models/caliop-random-forest-{date}.pkl", "rb") as f:
    classifier = pickle.load(f)

In [27]:
for c in classifier.estimators_:
    n_nodes = c.tree_.node_count
    max_depth = c.tree_.max_depth
    children_left = c.tree_.children_left
    children_right = c.tree_.children_right
    feature = c.tree_.feature
    threshold = c.tree_.threshold

    print(n_nodes, max_depth)

511 12
511 12
511 12
511 12
511 12


In [ ]:
X_testN, y_testN = get_xy(cloud_dfN)
X_testS, y_testS = get_xy(cloud_dfS)

y_predN = classifier.predict(X_testN)
y_predS = classifier.predict(X_testS)

y_pred_train = classifier.predict(X_train)


In [ ]:
def compute_error(y_pred, y_test):
 
    good_detected_cloud = np.sum(y_pred & y_test)
    wrong_detected_cloud = np.sum(y_pred & (~y_test))

    good_detected_clear = np.sum((~y_pred) & (~y_test))
    wrong_detected_clear = np.sum((~y_pred) & y_test)

    is_cloud = np.sum(y_test)
    is_not_cloud = np.sum(~y_test)

    return pd.DataFrame({'TPR': good_detected_cloud / is_cloud, 
                         'FPR': wrong_detected_cloud / is_not_cloud,
                         'TPR_clear': good_detected_clear / is_not_cloud,
                         'FPR_clear': wrong_detected_clear / is_cloud}, index=[0]) 

err = compute_error(y_pred, y_test)
err_train = compute_error(y_pred_train, y_train)
errN = compute_error(y_predN, y_testN)
errS = compute_error(y_predS, y_testS)

In [ ]:
err_ref0 = pd.read_csv("Errors/err-reference0.csv")
err_ref1 = pd.read_csv("Errors/err-reference1.csv")

plt.figure()
plt.plot(err.FPR, err.TPR, 'o', label='Random Forest (test dataset')
plt.plot(err_train.FPR, err_train.TPR, 'x', label='Random Forest (training dataset)')

plt.plot(errN.FPR, errN.TPR, 'x', markersize=10, label='Random Forest Greenland')
plt.plot(errS.FPR, errS.TPR, 'x', markersize=10, label='Random Forest Antarctic')
plt.plot(err_ref0.FPR, err_ref0.TPR, '-k', label='NN Ref0')
plt.plot(err_ref0.FPR, err_ref1.TPR, '-.k', label='NN Ref1')

plt.plot(0.132132, 0.81991, 's', label="SCDA Greenland")  #  	0.81991 	0.132132
plt.plot(0.147379, 0.882955, 's', label="SCDA Antarctique")
plt.grid(alpha=0.2)
plt.legend()
plt.xlabel("False positive")
plt.ylabel("True positive")
plt.title("Cloud detection")
